In [ ]:
using DrWatson
@quickactivate "masterarbeit"

In [ ]:
using BenchmarkTools
using ProgressMeter
using CUDA
using Flux
using LaTeXStrings
using Flux: train!
using GLMakie
using Printf
using Dates
using JLD2
using TOML
using StatsBase # for fit(histogram)

In [ ]:
Makie.inline!(true)
fontsize_theme = Theme(fontsize=35)
set_theme!(fontsize_theme)
wblue = Makie.wong_colors()[1]
worange = Makie.wong_colors()[2]
wgreen = Makie.wong_colors()[3]
wpink = Makie.wong_colors()[4]
wlblue = Makie.wong_colors()[5]
worange = Makie.wong_colors()[6]
wyellow = Makie.wong_colors()[7];

In [ ]:
using Revise

In [ ]:
using masterarbeit

In [ ]:
function jacobian6cl(m::Chain, cm::ChannelMapping, x::T) where {T <: AbstractArray{F}} where F<:Real
    sl1 = m[1]
    cl1 = m[2]
    sl2 = m[3]
    cl2 = m[4]
    sl3 = m[5]
    cl3 = m[6]
    sl4 = m[7]
    cl4 = m[8]
    sl5 = m[9]
    cl5 = m[10]
    sl6 = m[11]
    cl6 = m[12]

    x1s = sl1(x)
    x2 = cl1(x1s)
    x2s = sl2(x2)
    x3 = cl2(x2s)
    x3s = sl3(x3)
    x4 = cl3(x3s)
    x4s = sl4(x4)
    x5 = cl4(x4s)
    x5s = sl5(x5)
    x6 = cl5(x5s)
    x6s = sl6(x6)
    x7 = cl6(x6s)

    det1 = abs.(cldet(cl1, x1s[cl1.dimA+1:cl1.d,:], cl1.m(x1s[1:cl1.dimA,:])...))
    det2 = abs.(cldet(cl2, x2s[cl2.dimA+1:cl2.d,:], cl2.m(x2s[1:cl2.dimA,:])...))
    det3 = abs.(cldet(cl3, x3s[cl3.dimA+1:cl3.d,:], cl3.m(x3s[1:cl3.dimA,:])...)) 
    det4 = abs.(cldet(cl4, x4s[cl4.dimA+1:cl4.d,:], cl4.m(x4s[1:cl4.dimA,:])...)) 
    det5 = abs.(cldet(cl5, x5s[cl5.dimA+1:cl5.d,:], cl5.m(x5s[1:cl5.dimA,:])...)) 
    det6 = abs.(cldet(cl6, x6s[cl6.dimA+1:cl6.d,:], cl6.m(x6s[1:cl6.dimA,:])...)) 

    return abs(cmdet(cm)) .* det1 .* det2 .* det3 .* det4 .* det5 .* det6
end

In [ ]:
function lossf(m::Chain, cm::ChannelMapping, f::Function, x::T) where T<:AbstractArray{F} where F<:Real
    zi = cm(m(x))
    g = 1 ./ jacobian6cl(m, cm, x)
    fracs = abs.(f(zi) .- g) .^F(1.5) ./ f(zi)
    return sum(fracs) / size(x,2)
end

In [ ]:
f = double_gauss
ytozmap = IdentityMapping()
dim = 5;

In [ ]:
batchsize = 16384
N_epochs = 100
bins = 10
learning_rate = 0.01
decay = 0.7
optimizer = Adam
;

In [ ]:
function subnet(dimA::Signed, dimB::Signed, bins::Signed, width=10)
    return Chain(
        Split(
            Chain(
                BatchNorm(dimA),
                Dense(dimA => width, relu),
                Dense(width => width, relu),
                Dense(width => width, relu),
                Dense(width => dimB*(bins+1))  
                ), 
            Chain(
                BatchNorm(dimA),
                Dense(dimA => width, relu),
                Dense(width => width, relu),
                Dense(width => width, relu),
                Dense(width => dimB*bins)
                )
            ) 
        ) |> gpu
end

In [ ]:
model = Flux.f32(Chain(
    masterarbeit.MaskLayer([true, false, true, false, true]),
    CouplingLayer(dim, 3, bins, subnet),
    masterarbeit.MaskLayer([false, false, false, true, true]),
    CouplingLayer(dim, 2, bins, subnet),
    masterarbeit.MaskLayer([true, false, true, false, true]),
    CouplingLayer(dim, 2, bins, subnet),
    masterarbeit.MaskLayer([false, false, false, true, true]),
    CouplingLayer(dim, 3, bins, subnet),
    masterarbeit.MaskLayer([true, true, true, true, false]),
    CouplingLayer(dim, 1, bins, subnet),
    masterarbeit.MaskLayer([false, false, false, false, true]),
    CouplingLayer(dim, 4, bins, subnet),
) |> gpu);

In [ ]:
# first run to compile
xtest = CUDA.rand(dim,batchsize);

In [ ]:
model(xtest);

In [ ]:
f(ytozmap(xtest));

In [ ]:
lossf(model,ytozmap,f,xtest)

In [ ]:
Flux.withgradient(m-> lossf(m,ytozmap,f,xtest), model);

# Training

In [ ]:
losses = Float32[]

In [ ]:
for i in 1:3
    println("Training with learning rate  = $(learning_rate)")
    losses = train_NN(model, dim, lossf, losses, ytozmap, f, epochs=N_epochs, batchsize=batchsize, optimizer=optimizer, learning_rate=learning_rate)
    learning_rate = learning_rate * decay
end;

In [ ]:
losses[end]

In [ ]:
fig = Figure(size=(1500,1000))
ax = Axis(fig[1,1], xlabel="epoch", ylabel="loss", yscale=log10)
lines!(1:length(losses), losses, linewidth=3, color=wblue, label="loss")
n = 10
lines!(n:length(losses), moving_average(losses, n), linewidth=4, color=worange, label="$n epoch \n moving average")
fig[1,2] = Legend(fig, ax)
fig

In [ ]:
samples = sample_NN(model, ytozmap, dim, 10^7, batchsize);

In [ ]:
xticks = [0.0, 0.25, 0.5, 0.75, 1.0]
yticks = [0.0, 0.25, 0.5, 0.75, 1.0];

In [ ]:
histo = fit(Histogram, (samples[1,:], samples[2,:]), nbins=100)
histo_n = StatsBase.normalize(histo, mode=:pdf)
fig = Figure(size=(1000,1000), figure_padding=40)
ax = Axis(fig[1,1], xlabel=L"x_1", ylabel=L"x_2", xticks=xticks, yticks=yticks, aspect=1)
hm = heatmap!(histo.edges[1], histo.edges[2], histo_n.weights,  colorrange=(0,5))
fig[1, 2] = GridLayout(width = 20)
Colorbar(fig[1,3], hm, width=40)
save("doublegauss5d_nis_6cl_samples.png", fig)
fig

In [ ]:
function slice_samples(ind1, ind2, name1, name2)
    histo = fit(Histogram, (samples[ind1,:], samples[ind2,:]), nbins=100)
    histo_n = StatsBase.normalize(histo, mode=:pdf)
    fig = Figure(size=(1000,1000), figure_padding=40)
    ax = Axis(fig[1,1], xlabel=latexstring(name1), xticks=xticks, yticks=yticks, aspect=1)
    hidexdecorations!(ax, ticks=false)
    hideydecorations!(ax, ticks=false)
    heatmap!(histo.edges[1], histo.edges[2], histo_n.weights, colorrange=(0,5))
    save("doublegauss5d_nis_6cl_"*(name1)*"-"*(name2)*".png", fig)
    fig
end
slice_samples(1, 2, "x_1", "x_2")

In [ ]:
slice_samples(1, 2, "x_1", "x_2")
slice_samples(1, 3, "x_1", "x_3")
slice_samples(1, 4, "x_1", "x_4")
slice_samples(1, 5, "x_1", "x_5")
slice_samples(2, 3, "x_2", "x_3")
slice_samples(2, 4, "x_2", "x_4")
slice_samples(2, 5, "x_2", "x_5")
slice_samples(3, 4, "x_3", "x_4")
slice_samples(3, 5, "x_3", "x_5")
slice_samples(3, 5, "x_3", "x_5")
slice_samples(4, 5, "x_4", "x_5")

In [ ]:
wi_vegas = load_object("vegas_weights.jld2");

In [ ]:
function weights6cl(m::Chain, cm::ChannelMapping, f::Function, x::T) where {T <: AbstractArray}
    return jacobian6cl(m, cm, x) .* f(cm(m(x)))
end

function weights6cl_chunked(m, dim, cm, f, N, batchsize)
    if (N%batchsize != 0) 
        x = CUDA.rand(dim, N%batchsize)
        weights = weights6cl(m, cm, f, x)
        inputs = x
        runs = N ÷ batchsize 
    else
        x = CUDA.rand(dim,   batchsize)
        weights = weights6cl(m, cm, f, x)
        inputs = x
        runs = N ÷ batchsize - 1
    end
    for i in 1:runs
        x = CUDA.rand(dim, batchsize)
        weights = hcat(weights, weights6cl(m, cm, f, x))
        inputs = hcat(inputs, x)
    end
    return weights |> cpu, inputs
end

wi_m, x_for_wi = weights6cl_chunked(model, dim, ytozmap, f,  10^7, batchsize)

In [ ]:
wi = wi_m[1,:]
w_avg = mean(wi)
w_max = maximum(wi)
wi_n = wi ./ w_max
w_avg_n = mean(wi_n)
println("mean weight = $(mean(wi))")
println("max weight = $(maximum(wi))")
println("unweighting efficiency = $(mean(wi)/maximum(wi))")

In [ ]:
wi = wi_m[1,:]
f_evals = f(samples)[1,:]
f_over_g = wi
mcint = 0.0088
nis_int = sum(f_over_g) / size(samples,2)
mcerror = sqrt(sum((f_over_g  .- mcint).^2) / (size(samples,2)-1))
println("mc integral = $nis_int")
println("standard deviation = $mcerror")